# Multiple regression - the exercise

In [ ]:
# importing the modules we need

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# some extra modules for some useful types of plot
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Scipy minimize
from scipy.optimize import minimize

## Introduction

Let's import the data we need. This is the RateMyProfessors dataset we looked
at previously.

In [ ]:
df = pd.read_csv('rate_my_course.csv')
df.head()

We are going to perform two linear regressions in this exercise.

First, a simple linear regression (one predictor variable, one outcome
variable) - we will predict 'Clarity' from 'Helpfulness'.

Second, a multiple linear regression (several predictors, one outcome
variable) - we will predict 'Clarity' from 'Easiness' and 'Helpfulness'.

Create a dataframe called `df_reg` which contains only the variables we need
for these regressions (Easiness, Helpfulness and Clarity).

In [ ]:
df_reg = ...
# Show the first five rows.
df_reg.head()

## Simple Regression Exercise

Before performing any sort of regression analysis, it is sensible to visually
inspect the data.

Create a scatterplot showing the relationship between 'Helpfulness' and
'Clarity'.

In [ ]:
...

For reflection: from visual inspection do you think there is a linear
relationship between these variables?

Let's perform a simple linear regression to see if your judgment is correct.

Write a function, called `sos_error_for_minimize` which takes as its input a
list containing an intercept and a slope.

The function `sos_error_for_minimize` should use the intercept and slope to predict Clarity rating scores from the Helpfulness rating scores stored in `df_reg`. Remember: `prediction = intercept + slope * predictor`, where `predictor` is the variable you are trying to predict from.

`sos_error_for_minimize` should then calculate the prediction errors by
subtracting these predictions from the actual Clarity scores stored in
`df_reg`. It should return the sum of the squared prediction errors.

*Hint:* you may find it helpful to refer to the [Finding Lines page](https://matthew-brett.github.io/cfd2020/mean-slopes/finding_lines.html)

In [ ]:
def sos_error_for_minimize(intercept_and_slope):
    intercept = intercept_and_slope[0]
    slope = intercept_and_slope[1]
    ...

Run the cell below to check that your function is working.

In [ ]:
sos_error_for_minimize([1,1])

Now use minimize, from the scipy library, to minimize the value of
`sos_error_for_minimize`. Store the minimization result that comes back from
`minimize` in a variable called `simple_reg`.

*Hint:* again, you may find it helpful to refer to the [Finding Lines page](https://matthew-brett.github.io/cfd2020/mean-slopes/finding_lines.html)

In [ ]:
simple_reg = minimize(...)
# Show the result
simple_reg

## Using regression coefficients to generate predictions

The code in the cell below uses the intercept and slope - found by `minimize`
- to generate predicted Clarity scores from the Helpfulness scores. 

These predictions are then plotted along with the actual Clarity scores. You
can see that the predictions capture the general trend pretty well.

*Hint*: paying attention to the first three lines of this code may help you
later in the exercise...

In [ ]:
# getting the intercept value from the output from minimize
intercept = simple_reg.x[0]

# getting the slope value for the predictor from the output from minimize
slope = simple_reg.x[1]

# generating the predictions
predicted_values_simple_reg = intercept + df_reg['Helpfulness'] * slope

# generating the scatter plot
plt.scatter(df_reg['Helpfulness'], df_reg['Clarity'], label = 'actual values')
plt.scatter(df_reg['Helpfulness'], predicted_values_simple_reg, label = 'predicted values')
plt.xlabel('Helpfulness')
plt.ylabel('Clarity')
plt.legend();

## Moving to Multiple Regression

Predicting the values of one outcome variable from one predictor variable is
useful. However, often we are interested in the relationships between multiple
variables. This brings us to multiple regression.

Once again, before performing any analysis it is sensible to visually inspect
the data.

`pairplot()` from the [Seaborn plotting library](https://seaborn.pydata.org/)
is a useful function to inspect the relationship between multiple predictor
variables and one outcome variable. It was imported at the start of this
notebook using `import seaborn as sns`. This function might be useful to you
in your projects.

The cell below contains code which generates a *pairplot* which shows Number
of Professors and Clarity as the predictor (x) variables, and Overall Quality
as the outcome (y) variable.

In [ ]:
# Creating a pairplot in Seaborn
sns.pairplot(data = df,
             x_vars = ['Number of Professors', 'Clarity'],
             y_vars = 'Overall Quality');

You can see that, from visual inspection, it looks like there is a strong
linear relationship between Clarity and Overall Quality, but not between
Number of Professors and Overall Quality.

As mentioned earlier, for our multiple regression we are going to predict
Clarity from Easiness and Helpfulness.

Modify the `sns.pairplot()` code to generate a pairplot which uses 'Easiness' and 'Helpfulness' as the predictors (x_vars) and 'Clarity' as the outcome (y_vars).

In [ ]:
sns.pairplot(...)

For reflection: do you think Easiness and Helpfulness look like they have a
linear relationship with clarity?

To show the relationship further, run the code in the cell below to generate a
3D scatter plot.  Don't worry about the detail of the code below.

In [ ]:
# Run this code; please don't worry about the detail.
import mpl_toolkits.mplot3d
fig = plt.figure(figsize = (10,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_reg['Helpfulness'], df_reg['Easiness'], df_reg['Clarity'])
plt.xlabel('Helpfulness Rating')
plt.ylabel('Easiness Rating')
ax.set_zlabel('Clarity Rating');

## Multiple Regression

The functions in the three cells below give us the machinery we need for
multiple regression using `minimize`. They are modified from the [Simple and
multiple regression textbook page](https://matthew-brett.github.io/cfd2020/classification/single_multiple.html).

Read the docstring and the comments in the cells and make sure you understand
what each line of the function is doing.

**Remember to run all three cells.**


In [ ]:
def predict(intercept, slopes, row):
    """ Predict a value given an intercept, slopes and corresponding row values
    """
    return intercept + np.sum(slopes * np.array(row))

In [ ]:
def rmse(intercept, slopes, attributes, y_values):
    """ Root mean square error for prediction of `y_values` from `attributes`

    Use `intercept` and `slopes` multiplied by `attributes` to give prediction.

    `attributes` is a data frame with numerical attributes to predict from.
    """
    errors = []  # create an empty list, to store the prediction errors

    # for every observation in the dataset (`i` is the index of the current
    # observation)
    for i in np.arange(len(y_values)):

        # call the predict() function and apply it to the current row of the
        # attributes dataframe, using the intercept and slope values
        predicted = predict(intercept, slopes, attributes.iloc[i])

        # get the actual value of the outcome variable for this observation
        actual = y_values.iloc[i]

        # Calculate the squared prediction error for this observation and
        # append it to the 'errors' list.
        errors.append((predicted - actual) ** 2)

    # Return the square root of the average squared error
    return np.sqrt(np.mean(errors))

In [ ]:
def rmse_for_params(params):
    """ RMSE for intercept, slopes contained in `params`

    `params[0]` is the intercept.  `params[1:]` are the slopes.
    """
    # store the first value of the 'params' list in a variable called
    # 'intercept'
    intercept = params[0]

    # store the rest of the values in the 'params' list in a variable called
    # 'slopes'
    slopes = params[1:]

    # call the rmse() function and return its output
    return rmse(intercept,
                slopes,
                df_reg.loc[:, ['Easiness', 'Helpfulness']], # predictors
                df_reg['Clarity']  # outcome variable
               )

*Note*: this way of doing things takes the predictor variable values and the
outcome variable values from the 'top level' e.g. they are not given to the
`rmse_for_params()` function as arguments. Therefore, this function wouldn't
work if we were using different variables as the predictors or the outcome. We
would need to write another function, or use the method shown in the [multiple
regression page](https://matthew-brett.github.io/cfd2020/classification/Multiple_Regression).

Use `minimize` to perform multiple linear regression, using the
`rmse_for_params()` function. Predict Clarity from Easiness and Helpfulness.
Store the result from `minimize` in a variable called `min_multi_res`.

*Hint*: you may find it helpful to refer to the [Simple and multiple regression page](https://matthew-brett.github.io/cfd2020/classification/single_multiple)

In [ ]:
min_multi_res = ...
# Show the result.
min_multi_res

Now, get the values of the intercept and slopes and store these in separate
variables.

*Hint*:  you may want to refer to the cell above entitled 'Using regression
coefficients to generate predictions'.

In [ ]:
intercept = ...
easiness_slope = ...
helpfulness_slope = ...

Use the intercept and slope variables you have just created to generate
predicted Clarity scores from the Easiness and Helpfulness scores.

Remember - for multiple regression:

```
prediction = intercept + slope_1 * predictor_1 + slope_2 * predictor_2
```

In this case Easiness and Helpfulness are the predictors.

*Hint*: again, you may want to refer to the cell above entitled 'Using
regression coefficients to generate predictions'.

In [ ]:
predicted_values = intercept + ...
# Show the result.
predicted_values

The cell below plots the predicted values against the actual values. For
reflection: how well do you think multiple linear regression did at capturing
the general trend in the data?

In [ ]:
# Do not worry about this code, it just generates the 3D plot
fig = plt.figure(figsize = (10,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_reg['Helpfulness'], df_reg['Easiness'], df_reg['Clarity'], label = 'actual values')
ax.scatter(df_reg['Helpfulness'], df_reg['Easiness'], predicted_values, label = 'predicted values', color = 'red')
plt.legend()
plt.xlabel('Helpfulness Rating')
plt.ylabel('Easiness Rating')
ax.set_zlabel('Clarity Rating');

## Linear Regression with statsmodels

We can also perform linear regression using functions from the Statsmodels
library.

The cell below uses statsmodels to predict Clarity from Helpfulness, using
linear regression. *Note*: this is the same simple linear regression we did
earlier...

In [ ]:
import statsmodels.formula.api as smf

simple_model_sm = smf.ols(formula="Clarity ~ Helpfulness", data = df_reg) # create a model
simple_fit_sm = simple_model_sm.fit() # fit the model
simple_fit_sm.summary() # show the model summary

Look at the intercept and slope values from statsmodels (shown in the table
above under 'coef').

Let's compare those values to the intercept and slope values we got from our
simple linear regression, where we predicted Clarity from Helpfulness using
`minimize`:

In [ ]:
simple_reg.x

How similar/different are the slope and intercept values that Statsmodels
found from the ones we found using `minimize`? Write your answer in the cell
below:

<i> Your answer here... </i>


## Multiple Regression with Statsmodels

Let's perform the same multiple regression we did above, but this time using
Statsmodels.

Expanding on the statsmodels code presented above, use statsmodels to predict
Clarity from Easiness and Helpfulness. Call the model `multi_model_sm` and
called the fitted model `multi_fit_sm`.

*Hint*: you may want to refer to the [Simple and multiple regression textbook
page](https://matthew-brett.github.io/cfd2020/classification/single_multiple.html).

In [ ]:
multi_model_sm = ...
multi_fit_sm = ...
# Show the summary.
multi_fit_sm.summary()

Again, look at the intercept and slope values from statsmodels (shown in the
table above under 'coef').

Let's compare those values to the intercept and slope values we got from our
multiple linear regression, where we predicted Clarity from Easiness and
Helpfulness using `minimize`:

In [ ]:
min_multi_res.x

How similar/different are the slope and intercept values that statsmodels
found from the ones we found using `minimize`? Write your answer in the cell
below:

*Your answer here...*

## Comparing statsmodels predictions to minimize

Just for fun, let's see how close the predictions are from each method of
performing multiple linear regression (`minimize` vs Statsmodels).

The `.predict()` method from Statsmodels can be used to generate predictions
from a model. The cell below uses this method to predict Easiness and
Helpfulness from Clarity.

In [ ]:
attributes = df_reg[['Easiness', 'Helpfulness']]
sm_predictions = multi_fit_sm.predict(attributes)
sm_predictions

Let's take these predicted Clarity values from Statsmodels, and subtract them
from the predictions we got from multiple regression using minimize.

If these values are small, it means the predictions from the two methods were
very close.

In [ ]:
minimize_vs_sm = predicted_values - sm_predictions
minimize_vs_sm

Plot a histogram of the differences in the predictions between the two methods.

In [ ]:
...

Calculate the mean difference between the predictions from the two methods:

In [ ]:
mean_difference_in_prediction = np.mean(minimize_vs_sm)
# Show the result.
mean_difference_in_prediction

Finally, use `np.count_nonzero()` to count the number of differences in the
predictions between the two methods which have an absolute value larger than
0.001.

In [ ]:
np.count_nonzero(...)

Do you think the models are generating similar or different predictions? Why?
Write your answer in the cell below:

*Your answer here...*